In [1]:
import functools
import os
import math

from pathlib import Path
from tqdm import tqdm

from absl import app
from absl import flags

import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from model.model import DelfArcFaceModel
from utils.preprocessing import CreateDataset


In [2]:
batch_size = 64
image_size = 224
STEPS_PER_TPU_CALL = 1
learning_rate = 5e-5  # should be smaller than training on single GPU
feature_size = 2048  # Embedding size before the output layer
save_interval = 2000

# ArcFace params
margin = 0.1  # DELG used 0.1, original ArcFace paper used 0.5. When margin is 0, it should be the same as doing a normal softmax but with embedding and weight normalised.
logit_scale = int(math.sqrt(feature_size))

# GeM params
gem_p = 3.
train_p = False  # whether to learn gem_p or not
data_dirs = '/home/ubuntu/Dacon/cpt_data/landmark/'
data_dir = "/home/ubuntu/Dacon/jin/NIA"

In [3]:
checkpoint_dir = "/home/ubuntu/Dacon/jin/NIA/checkpoint/"
test_tf_records_dir = "/home/ubuntu/Dacon/jin/NIA/tfrecords_test/train*"
training_csv_path = os.path.join(data_dir, "train.csv")
train_csv = pd.read_csv(str(training_csv_path))
num_samples = len(train_csv["id"].tolist())
unique_landmark_ids = train_csv["landmark_id"].unique().tolist()
unique_landmark_ids = tf.convert_to_tensor(unique_landmark_ids, dtype=tf.int64)

In [4]:
test_set = CreateDataset(test_tf_records_dir)

In [5]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [6]:
test_data = test_datagen.flow_from_directory(
    directory = data_dirs + 'test/',
    class_mode = 'sparse',
    shuffle = True,
    target_size = (image_size, image_size),
    batch_size = batch_size)

Found 7488 images belonging to 309 classes.


In [7]:
eval_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(
      name='eval_accuracy')

new_model = DelfArcFaceModel(
            input_shape=(image_size, image_size, 3), n_classes=len(unique_landmark_ids), margin=margin, logit_scale=logit_scale,
            p=gem_p, train_p=train_p, feature_size=feature_size
        )
new_optimizer = tf.keras.optimizers.Adam()



In [8]:
@tf.function
def eval_step(images, labels):
  predictions = new_model((images, labels), training=False)
  eval_accuracy(labels, predictions)

In [9]:
checkpoint = tf.train.Checkpoint(optimizer=new_optimizer, model=new_model)
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [10]:
for images, labels in tqdm(test_data):
  eval_step(images, labels)
  print(eval_accuracy.result()*100)

  1%|          | 1/117 [00:05<11:18,  5.85s/it]

tf.Tensor(0.0, shape=(), dtype=float32)


  2%|▏         | 2/117 [00:06<08:08,  4.24s/it]

tf.Tensor(0.0, shape=(), dtype=float32)


  3%|▎         | 3/117 [00:06<05:56,  3.13s/it]

...

tf.Tensor(0.0, shape=(), dtype=float32)


  5%|▌         | 6/117 [00:08<02:34,  1.39s/it]

tf.Tensor(0.0, shape=(), dtype=float32)


  5%|▌         | 6/117 [00:08<02:43,  1.47s/it]


KeyboardInterrupt: 

In [26]:
next(iter(test_set))[1]

<tf.Tensor: shape=(32,), dtype=int64, numpy=
array([ 16, 117, 157,  86, 157, 117,  86,  86,  86, 157,  86, 117,  16,
       157, 157, 117,  86,  16, 157,  86,  16, 157, 117,  91, 117,  86,
        86, 157,  91, 117, 157, 157])>